#### How deepchecks detects outliers

Outlier Sample Detection searches for outliers samples (jointly across all features) using the LoOP algorithm. The LoOP algorithm is a robust method for detecting outliers in a dataset across multiple variables by comparing the density in the area of a sample with the densities in the areas of its nearest neighbors (see link for further details).

LoOP relies on a distance matrix. In our implementation we use the Gower distance that averages the distances per feature between samples. For numeric features it calculates the absolute distance divided by the range of the feature and for categorical features it is an indicator whether the values are the same (see link for further details).

In [1]:
import pandas as pd
from deepchecks.tabular.datasets.classification.phishing import load_data
from deepchecks.tabular import Dataset
from deepchecks.tabular.checks.integrity.outlier_sample_detection import OutlierSampleDetection

#### Load Dataset

In [2]:
phishing_dataset = load_data(as_train_test=False, data_format='DataFrame')
phishing_dataset

,target,month,scrape_date,ext,urlLength,numDigits,numParams,num_%20,num_@,entropy,...,dse,bodyLength,numTitles,numImages,numLinks,specialChars,scriptLength,sbr,bscr,sscr
0,0,1,2019-01-01,net,102,8,0,0,0,-4.384032,...,191,32486,3,5,330,9419,23919,0.736286,0.289940,2.539442
1,0,1,2019-01-01,country,154,60,0,2,0,-3.566515,...,0,16199,0,4,39,2735,794,0.049015,0.168838,0.290311
2,0,1,2019-01-01,net,171,5,11,0,0,-4.608755,...,104,103344,18,9,302,27798,83817,0.811049,0.268985,2.412174
3,0,1,2019-01-01,com,94,10,0,0,0,-4.548921,...,466,34093,11,43,199,9087,19427,0.569824,0.266536,2.137889
4,0,1,2019-01-01,other,95,11,0,0,0,-4.717188,...,928,202,1,0,0,39,0,0.000000,0.193069,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11345,0,1,2020-01-15,country,89,7,0,0,0,-4.254491,...,0,4117,5,0,1,971,1866,0.625302,0.213266,2.932029
11346,0,1,2020-01-15,other,107,13,0,0,0,-4.758879,...,1882,17788,47,58,645,3185,4228,0.291069,0.214348,1.357928
11347,0,1,2020-01-15,com,112,10,0,0,0,-4.723014,...,1011,0,0,0,0,0,0,0.000000,0.000000,0.000000
11348,0,1,2020-01-15,html,111,3,0,0,0,-4.289384,...,265,0,0,0,0,0,0,0.000000,0.000000,0.000000


In [3]:
phishing_dataset.describe()

,target,month,urlLength,numDigits,numParams,num_%20,num_@,entropy,has_ip,dsr,dse,bodyLength,numTitles,numImages,numLinks,specialChars,scriptLength,sbr,bscr,sscr
count,11350.000000,11350.00000,11350.000000,11350.000000,11350.000000,11350.000000,11350.000000,11350.000000,11350.0,11350.000000,11350.000000,1.135000e+04,11350.000000,11350.000000,11350.000000,11350.000000,1.135000e+04,11350.000000,11350.000000,11350.000000
mean,0.030837,6.28000,114.025991,14.304053,0.523172,0.028811,0.005286,-4.480238,0.0,3918.588106,439.518414,4.956919e+04,11.489251,15.101410,97.996828,11413.852423,3.158685e+04,0.455681,0.211830,3.668419
std,0.172884,3.55003,43.531701,19.219679,1.511591,0.325926,0.078358,0.304459,0.0,3593.174607,695.881955,1.317944e+05,31.267348,38.472225,164.578212,32281.711555,1.097909e+05,0.374175,0.103761,19.381514
min,0.000000,1.00000,37.000000,0.000000,0.000000,0.000000,0.000000,-5.902860,0.0,0.000000,-1277.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,0.000000,3.00000,91.000000,4.000000,0.000000,0.000000,0.000000,-4.630107,0.0,0.000000,0.000000,1.940000e+02,1.000000,0.000000,0.000000,35.000000,0.000000e+00,0.000000,0.181818,0.000000
50%,0.000000,6.00000,102.000000,8.000000,0.000000,0.000000,0.000000,-4.488220,0.0,3819.000000,182.000000,1.266750e+04,1.000000,2.000000,24.000000,2707.000000,4.647000e+03,0.494361,0.244586,1.721923
75%,0.000000,9.00000,120.000000,18.000000,0.000000,0.000000,0.000000,-4.321382,0.0,7078.000000,387.000000,3.956700e+04,11.000000,13.000000,140.000000,9434.750000,2.033550e+04,0.788890,0.271936,2.747405
max,1.000000,12.00000,772.000000,454.000000,12.000000,12.000000,2.000000,-3.165609,0.0,11391.000000,3643.000000,2.043278e+06,1042.000000,642.000000,1235.000000,506888.000000,1.988078e+06,1.001634,0.617100,368.420245


#### Outlier Check

In [4]:
check = OutlierSampleDetection(nearest_neighbors_percent=0.01, extent_parameter=3)
check.run(phishing_dataset)

Received a "pandas.DataFrame" instance, initializing "deepchecks.tabular.Dataset" from it
It is recommended to initialize Dataset with categorical features by doing "Dataset(df, cat_features=categorical_list)". No categorical features were passed, therefore heuristically inferring categorical features in the data.
10 categorical features were inferred: target, month, ext, numParams, num_%20, num_@, has_ip... For full list use dataset.cat_features


#### Define a condition

In [5]:
check = OutlierSampleDetection()
check.add_condition_outlier_ratio_not_greater_than(max_outliers_ratio=0.001, outlier_score_threshold=0.9)
check.run(phishing_dataset)

Received a "pandas.DataFrame" instance, initializing "deepchecks.tabular.Dataset" from it
It is recommended to initialize Dataset with categorical features by doing "Dataset(df, cat_features=categorical_list)". No categorical features were passed, therefore heuristically inferring categorical features in the data.
10 categorical features were inferred: target, month, ext, numParams, num_%20, num_@, has_ip... For full list use dataset.cat_features
